---
title: "A Replication of Karlan and List (2007)"
author: "Lebin Sun"
date: April 23, 2025
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---


## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a field experiment to test the effectiveness of different fundraising letters. They sent out 50,000 fundraising letters to potential donors, randomly assigning each letter to one of three treatments: a standard letter, a matching grant letter, or a challenge grant letter. They published the results of this experiment in the _American Economic Review_ in 2007. The article and supporting data are available from the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action as part of [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).

This study investigate whether the presence and size of a matching grant—commonly used by nonprofits—significantly affects donor behavior. The matching grant acts as a potential price reduction for donating, which may alter individuals’ willingness to give. The experiment’s large sample and randomized design make it especially well-suited to test economic theories of altruism, incentives, and social signaling.

This project seeks to replicate the core findings of Karlan and List (2007), using the original dataset provided by the authors. Through the replication, I aim to confirm their main results, including how matching grants affect both the likelihood of donating and the average amount donated. I also explore potential heterogeneity in treatment effects by political geography, as highlighted in the original paper.

## Data

### Description

This dataset comes from a large-scale natural field experiment conducted by Karlan and List (2007) to study the impact of different fundraising strategies on charitable donations. It contains data from over 50,000 past donors to a U.S.-based nonprofit organization who were randomly assigned to receive one of several types of solicitation letters.

Each observation in the dataset corresponds to a single individual donor. The dataset includes variables that capture:

    - Treatment assignment including whether the donor received a matching grant offer and the specific match ratio (1:1, 2:1, or 3:1).
    - Suggested donation amounts which were manipulated as part of the experimental design.
    - Donation behavior such as whether the person donated in response, and how much they gave.
    - Demographic indicators and prior donation history (e.g., years since first gift, total number of past donations, gender).
    - Geographic and political indicators such as whether the donor lived in a “red” or “blue” state during the 2004 presidential election, and the level of activity of the organization in that state.

The following Python code chunk loads the original dataset Dean Karlan used: 

In [ ]:
import pandas as pd
df = pd.read_stata("karlan_list_2007.dta")

### Data Overview
This following table provides an overview of all variables in the dataset. It includes each variable’s name, its data type (e.g., numeric or categorical), and the number of missing values.

In [ ]:
#| echo: true
#| code-fold: true
overview_table = pd.DataFrame({
    "Variable": df.columns,
    "Data Type": df.dtypes.values,
    "Missing Values": df.isnull().sum().values
})

overview_table

### Variable Definitions
The summary statistics table below presents descriptive metrics for all numeric variables in this dataset, including count, mean, standard deviation, and range (min/max).

In [ ]:
#| echo: true
#| code-fold: true
df.describe().transpose().round(2)

:::: {.callout-note collapse="true"}
### Variable Definitions

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::


### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.

In [ ]:
#| echo: true
#| code-fold: true
import numpy as np
import pandas as pd

treated = df[df['treatment'] == 1]
control = df[df['treatment'] == 0]

table1_vars = [
    'mrm2',    
    'hpa',      
    'freq',      
    'years',     
    'year5',     
    'female', 'couple',
    'redcty',
    'nonlit', 'cases'
]

def manual_ttest(var):
    Xt, Xc = treated[var], control[var]
    mean_diff = Xt.mean() - Xc.mean()
    pooled_se = np.sqrt(Xt.var(ddof=1)/len(Xt) + Xc.var(ddof=1)/len(Xc))
    t_stat = mean_diff / pooled_se
    return {
        "Variable": var,
        "Mean_Treatment": round(Xt.mean(), 3),
        "Mean_Control": round(Xc.mean(), 3),
        "t-stat": round(t_stat, 4)
    }

manual_ttest_df = pd.DataFrame([manual_ttest(v) for v in table1_vars])
manual_ttest_df

This table replicates the balance check presented in Table 1 of Karlan & List (2007), using a manual t-test formula to compare pre-treatment characteristics across the treatment and control groups. The variables cover donor behavior (e.g., months since last donation, prior contributions), demographics (e.g., gender, race, household size), and political/geographic indicators (e.g., red state/county, legal involvement of the organization). Across all tested variables, the t-statistics remain small (generally below ±2), and none of the differences reach conventional levels of statistical significance. This provides strong evidence that the treatment assignment was effectively randomized and groups are balanced on observable characteristics. This confirms the reliability of the experiment design and supports the validity of later comparisons on donation behavior.

In [ ]:
#| echo: true
#| code-fold: true
# from tabulate import tabulate
import statsmodels.formula.api as smf

reg_results = []
for var in table1_vars:
    model = smf.ols(f"{var} ~ treatment", data=df).fit()
    reg_results.append([
        var,
        round(model.params["treatment"], 4),
        round(model.pvalues["treatment"], 4),
        round(model.rsquared, 4)
    ])

# headers = ["Variable", "Treatment Coef", "p-value", "R-squared"]
# print(tabulate(reg_results, headers=headers, tablefmt="github"))
reg_df = pd.DataFrame(reg_results)
reg_df  

This regression-based balance check complements the earlier t-tests by estimating the relationship between treatment assignment and baseline covariates. For each variable, we regress it on the `treatment` indicator and inspect the coefficient and p-value. All p-values are well above the 0.05 threshold, confirming that treatment status is not significantly associated with any pre-treatment variable. This is consistent with the design of a randomized controlled trial, further supporting the internal validity of the experiment. The results also match those from the manual t-tests, which is expected since the regression with a binary independent variable produces the same mean difference and inference as a t-test.


## Experimental Results

### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 

In [ ]:
#| echo: true
#| code-fold: true
import matplotlib.pyplot as plt

donation_rates = df.groupby("treatment")["gave"].mean().rename({0: "Control", 1: "Treatment"})

donation_rates.plot(kind="bar", color=["green", "orange"])
plt.title("Proportion of People Who Donated")
plt.ylabel("Donation Rate")
plt.xticks(rotation=0)
plt.ylim(0, 0.035)  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

This plot displays the proportion of individuals who made a charitable contribution in the control and treatment groups. The donation rate is noticeably higher in the treatment group, indicating that the presence of a matching donation offer likely increased the likelihood of giving. This supports the hypothesis that matched donations can serve as an effective behavioral nudge in charitable fundraising.

In [ ]:
#| echo: true
#| code-fold: true
from scipy.stats import ttest_ind

treated = df[df['treatment'] == 1]
control = df[df['treatment'] == 0]

t_stat, p_val = ttest_ind(treated["gave"], control["gave"], equal_var=False)

print(f"t-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")

In [ ]:
#| echo: true
#| code-fold: true
import statsmodels.formula.api as smf

model = smf.ols("gave ~ treatment", data=df).fit()

from tabulate import tabulate
table = [["Coef", model.params["treatment"]],
         ["p-value", model.pvalues["treatment"]],
         ["R-squared", model.rsquared]]
# print(tabulate(table, headers=["Metric", "Value"], tablefmt="github"))
df_table = pd.DataFrame(table)
df_table 

Both the t-test and linear regression show a statistically significant difference in donation behavior between the treatment and control groups. Individuals who received a letter mentioning a matching grant were more likely to donate than those who received a standard letter. This finding replicates Table 2a Panel A of Karlan & List (2007), and supports the hypothesis that matched donations act as a strong psychological motivator. From a behavioral standpoint, it suggests that people are more inclined to give when their contributions feel amplified — they may view their gift as having a greater impact, which encourages them to take action. This simple intervention—adding a match offer—significantly influenced behavior even though the individuals in both groups received similar messages aside from that detail.

In [ ]:
#| echo: true
#| code-fold: true
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tabulate import tabulate

probit_model = smf.probit("gave ~ treatment", data=df).fit(disp=0)

probit_summary = [
    ["Coef", probit_model.params["treatment"]],
    ["p-value", probit_model.pvalues["treatment"]],
    ["Pseudo R-squared", probit_model.prsquared]
]
#print(tabulate(probit_summary, headers=["Metric", "Value"], tablefmt="github"))
df_probit_summary = pd.DataFrame(probit_summary)
df_probit_summary

This probit regression estimates the effect of treatment assignment on the likelihood of making a donation, using a nonlinear model suitable for binary outcomes. The coefficient on the treatment variable is positive and statistically significant, indicating that individuals who received a matching gift message were more likely to donate. This result replicates the findings of Table 3, Column 1 in *Karlan & List (2007)*, providing further evidence that matching donations increase participation. While the probit coefficient itself isn’t directly interpretable as a percentage change, its sign and significance confirm the effect observed in the linear model and t-test. Together, these consistent results across models support the conclusion that match framing can significantly increase donor engagement in charitable campaigns.


### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

In [ ]:
#| echo: true
#| code-fold: true
from scipy.stats import ttest_ind

match_only = df[df["treatment"] == 1]

ratio1 = match_only[match_only["ratio"] == "ratio1"]
ratio2 = match_only[match_only["ratio"] == "ratio2"]
ratio3 = match_only[match_only["ratio"] == "ratio3"]

tests = {
    "2:1 vs 1:1": ttest_ind(ratio2["gave"], ratio1["gave"], equal_var=False),
    "3:1 vs 1:1": ttest_ind(ratio3["gave"], ratio1["gave"], equal_var=False),
    "3:1 vs 2:1": ttest_ind(ratio3["gave"], ratio2["gave"], equal_var=False),
}

import pandas as pd
ttest_results = pd.DataFrame([
    [k, round(v[0], 4), round(v[1], 4)]
    for k, v in tests.items()
], columns=["Comparison", "t-statistic", "p-value"])

ttest_results

The t-tests above examine whether increasing the match ratio (from 1:1 to 2:1 or 3:1) significantly affects the probability of donating. The results show that the differences in donation rates are not statistically significant** at conventional levels. This finding aligns with the authors' comment on page 8 of Karlan & List (2007), where they state that "the figures suggest that higher match ratios did not lead to significantly greater giving rates." Despite intuitive expectations, offering a 2:1 or 3:1 match did not lead to more people donating compared to a 1:1 match. This suggests that the psychological impact of having any match available may be more important than the generosity of the multiplier itself.

In [ ]:
#| echo: true
#| code-fold: true
import statsmodels.formula.api as smf
from tabulate import tabulate

matched = df[df["treatment"] == 1]

model = smf.ols("gave ~ C(ratio)", data=matched).fit()

summary_table = [
    ["Intercept (ratio1)", model.params["Intercept"], model.pvalues["Intercept"]],
    ["ratio2", model.params.get("C(ratio)[T.ratio2]", 0), model.pvalues.get("C(ratio)[T.ratio2]", 1)],
    ["ratio3", model.params.get("C(ratio)[T.ratio3]", 0), model.pvalues.get("C(ratio)[T.ratio3]", 1)],
]
#print(tabulate(summary_table, headers=["Term", "Coef", "p-value"], tablefmt="github"))
df_summary_table = pd.DataFrame(summary_table)
df_summary_table

This regression estimates the donation probability under different match ratios, using `ratio1` (1:1 match) as the baseline category. The intercept represents the average donation rate for the 1:1 group, while the coefficients for `ratio2` and `ratio3` represent the change in probability relative to that baseline. The results confirm that there are no statistically significant differences in donation rates when comparing 2:1 or 3:1 match ratios to the 1:1 ratio. These coefficients are close to zero and accompanied by large p-values, reinforcing the earlier t-test findings. In line with the authors' interpretation, this suggests that increasing the match ratio beyond 1:1 does not further motivate giving** — the existence of a match alone may be the primary driver of behavioral change, rather than its size.

In [ ]:
#| echo: true
#| code-fold: true
coef_ratio1 = model.params["Intercept"]
coef_ratio2 = model.params.get("C(ratio)[T.ratio2]", 0)
coef_ratio3 = model.params.get("C(ratio)[T.ratio3]", 0)

diff_2vs1 = coef_ratio2
diff_3vs2 = coef_ratio3 - coef_ratio2

import pandas as pd
response_diff = pd.DataFrame({
    "Comparison": ["2:1 vs 1:1", "3:1 vs 2:1"],
    "Difference in Fitted Rate": [round(diff_2vs1, 5), round(diff_3vs2, 5)]
})

response_diff

This table shows the difference in predicted donation rates based on the fitted values from the regression model.
    - The difference between the 2:1 and 1:1 match groups is very small and not statistically significant.
    - The difference between the 3:1 and 2:1 groups is similarly negligible.

These findings provide further evidence that increasing the match ratio beyond 1:1 does not meaningfully increase the likelihood of donating. The presence of a match appears to matter, but its size does not — supporting the idea that the psychological nudge of a match is more about its existence than its magnitude. This aligns with the interpretation found in *Karlan & List (2007)*, where the authors argue that the offer of any match (even 1:1) seems sufficient to trigger the intended behavioral response.


### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.

In [ ]:
#| echo: true
#| code-fold: true
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(treated["amount"], control["amount"], equal_var=False)
print(f"t-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")

In [ ]:
#| echo: true
#| code-fold: true
import statsmodels.formula.api as smf
from tabulate import tabulate

model_amt = smf.ols("amount ~ treatment", data=df).fit()

summary_table = [
    ["Intercept (Control)", model_amt.params["Intercept"], model_amt.pvalues["Intercept"]],
    ["Treatment Coef", model_amt.params["treatment"], model_amt.pvalues["treatment"]],
    ["R-squared", model_amt.rsquared, ""]
]
# print(tabulate(summary_table, headers=["Term", "Coef", "p-value"], tablefmt="github"))
df_summary_table = pd.DataFrame(summary_table)
df_summary_table

This analysis tests whether the average donation amount differs between the treatment and control groups — regardless of whether someone donated or not (i.e., this includes all the zeros). Both the t-test and linear regression show that the mean donation amount is slightly higher in the treatment group, but the difference is small in magnitude and statistically weak. The low R-squared value also indicates that treatment status explains very little of the variance in donation amounts. This suggests that while matched donations may influence whether people give, they don’t strongly influence how much they give, at least when considering all individuals (including those who gave $0). To better understand donation behavior among actual donors, we’ll refine this analysis next by limiting it to individuals who did donate.

In [ ]:
#| echo: true
#| code-fold: true
donors_only = df[df["gave"] == 1]

model_conditional = smf.ols("amount ~ treatment", data=donors_only).fit()

summary_table = [
    ["Intercept (Control)", model_conditional.params["Intercept"], model_conditional.pvalues["Intercept"]],
    ["Treatment Coef", model_conditional.params["treatment"], model_conditional.pvalues["treatment"]],
    ["R-squared", model_conditional.rsquared, ""]
]
#print(tabulate(summary_table, headers=["Term", "Coef", "p-value"], tablefmt="github"))
df_summary_table = pd.DataFrame(summary_table)
df_summary_table

By focusing only on those who actually donated, this regression estimates how much more (or less) people gave if they were in the treatment group compared to the control group. The estimated coefficient on `treatment` now reflects the difference in average gift size conditional on donating. The result shows that the treatment has very little effect on the amount donated among those who gave something — the coefficient is small and statistically insignificant. This finding supports the idea that the matching grant mainly affects the decision to donate, not the amount donated once that decision is made. In other words, treatment increases the extensive margin (whether to give), but not the intensive margin (how much to give). Regarding causal interpretation: since treatment was randomly assigned, and we're conditioning on an outcome (`gave`), the estimate does not have a strict causal interpretation. It's subject to selection bias — those who gave in treatment might be different in unobservable ways from those who gave in control. So we interpret this descriptively rather than causally.

In [ ]:
#| echo: true
#| code-fold: true
import matplotlib.pyplot as plt

donors_only = df[df["gave"] == 1]

treat_donors = donors_only[donors_only["treatment"] == 1]["amount"]
control_donors = donors_only[donors_only["treatment"] == 0]["amount"]

mean_treat = treat_donors.mean()
mean_control = control_donors.mean()

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Treatment plot
axes[0].hist(treat_donors, bins=30, color='orange', edgecolor='black', alpha=0.8)
axes[0].axvline(mean_treat, color='red', linestyle='--', label=f'Mean = {mean_treat:.2f}')
axes[0].set_title("Treatment Group")
axes[0].set_xlabel("Donation Amount ($)")
axes[0].set_ylabel("Number of Donors")
axes[0].legend()

# Control plot
axes[1].hist(control_donors, bins=30, color='green', edgecolor='black', alpha=0.8)
axes[1].axvline(mean_control, color='red', linestyle='--', label=f'Mean = {mean_control:.2f}')
axes[1].set_title("Control Group")
axes[1].set_xlabel("Donation Amount ($)")
axes[1].legend()

fig.suptitle("Histogram of Donation Amounts (Among Donors Only)", fontsize=14)
plt.tight_layout()
plt.show()

These histograms visualize the distribution of donation amounts among those who gave, separately for the treatment and control groups. The red vertical lines represent the sample average donation in each group. Below are the main observations: 
    - Both distributions are highly right-skewed with many small donations and a few large outliers.
    - The average donation amount is slightly lower in the treatment group than in the control group.
These observations align with earlier regression findings showing that while matched gifts increase the likelihood of donating, they do not increase the amount given by those who choose to donate. This suggests that matching gifts influence whether people give, but not how much they give, reinforcing the psychological interpretation that matched gifts are more about prompting action than amplifying generosity.


## Simulation Experiment

As a reminder of how the t-statistic "works," in this section I use simulation to demonstrate the Law of Large Numbers and the Central Limit Theorem.

Suppose the true distribution of respondents who do not get a charitable donation match is Bernoulli with probability p=0.018 that a donation is made. 

Further suppose that the true distribution of respondents who do get a charitable donation match of any size  is Bernoulli with probability p=0.022 that a donation is made.

In [ ]:
#| echo: true
#| code-fold: true
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

p_control = 0.018
p_treat = 0.022
n = 1000             
iterations = 10000   

t_stats = []

for _ in range(iterations):
    control_sample = np.random.binomial(1, p_control, n)
    treat_sample = np.random.binomial(1, p_treat, n)
    t_stat, _ = ttest_ind(treat_sample, control_sample, equal_var=False)
    t_stats.append(t_stat)

plt.figure(figsize=(10, 5))
plt.hist(t_stats, bins=50, color='orange', edgecolor='black', density=True)
plt.axvline(np.mean(t_stats), color='red', linestyle='--', label=f"Mean t = {np.mean(t_stats):.3f}")
plt.axvline(1.96, color='gray', linestyle=':', label='95% Critical Value')
plt.axvline(-1.96, color='gray', linestyle=':')
plt.title("Simulated Distribution of t-Statistics")
plt.xlabel("t-statistic")
plt.ylabel("Density")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

This simulation mimics running 10,000 randomized experiments comparing donation rates between a control group (with `p = 0.018`) and a treatment group (with `p = 0.022`), each with 1,000 individuals. I calculate the t-statistic for the difference in proportions in each simulated trial. The resulting histogram approximates the sampling distribution of the t-statistic under these parameters. The following are the key observations based on the simulation result:
    - The distribution is centered slightly above 0, reflecting the small true difference in means (0.004).
    - It approximates a normal distribution, which is expected from the Central Limit Theorem.
    - The proportion of simulated t-statistics exceeding ±1.96 shows how often a standard hypothesis test would reject the null of no difference at the 5% level.

This visually demonstrates how small but real differences can become detectable with sufficient sample size and repeated sampling — a core insight from the Law of Large Numbers and Central Limit Theorem.



### Law of Large Numbers

In [ ]:
#| echo: true
#| code-fold: true
import numpy as np
import matplotlib.pyplot as plt

n = 10000
p_control = 0.018
p_treat = 0.022

control = np.random.binomial(1, p_control, n)
treatment = np.random.binomial(1, p_treat, n)

diff = treatment - control
cumulative_avg = np.cumsum(diff) / np.arange(1, n + 1)
true_diff = p_treat - p_control

plt.figure(figsize=(10, 5))
plt.plot(cumulative_avg, color='orange', lw=2, label="Cumulative Average")
plt.axhline(true_diff, color='red', linestyle='--', label=f"True Diff = {true_diff:.4f}")
plt.title("Law of Large Numbers: Convergence of Mean Differences")
plt.xlabel("Number of Observations")
plt.ylabel("Cumulative Difference (Treatment - Control)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

This plot illustrates how the cumulative average difference in donation outcomes between treatment and control groups converges to the true difference in means (0.004) as more data is accumulated. Each point on the blue line shows the average difference in outcomes after that many paired observations. As the sample size increases, the average stabilizes — a clear demonstration of the Law of Large Numbers in action. The dashed red line marks the true difference in the population, and we see that the sample-based average gets increasingly close to it. This reflects why larger experiments give more reliable, less variable estimates.



### Central Limit Theorem

In [ ]:
#| echo: true
#| code-fold: true
import numpy as np
import matplotlib.pyplot as plt

p_control = 0.018
p_treat = 0.022
true_diff = p_treat - p_control
sample_sizes = [50, 200, 500, 1000]
iterations = 1000

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs = axs.flatten()

for i, n in enumerate(sample_sizes):
    mean_diffs = []
    for _ in range(iterations):
        ctrl = np.random.binomial(1, p_control, n)
        treat = np.random.binomial(1, p_treat, n)
        mean_diffs.append(np.mean(treat) - np.mean(ctrl))
    
    axs[i].hist(mean_diffs, bins=30, color='orange', edgecolor='black')
    axs[i].axvline(true_diff, color='red', linestyle='--', label=f"True diff = {true_diff:.4f}")
    axs[i].axvline(0, color='black', linestyle=':', label="Zero")
    axs[i].set_title(f"Sample Size = {n}")
    axs[i].legend()
    axs[i].set_xlabel("Sample Mean Differences")
    axs[i].set_ylabel("Frequency")
    axs[i].grid(True, linestyle='--', alpha=0.5)

fig.suptitle("Central Limit Theorem: Distribution of Sample Mean Differences", fontsize=14)
plt.tight_layout()
plt.show()

Each histogram shows the distribution of sample mean differences between treatment and control groups from 1000 repeated experiments. The sample size increases from 50 to 1000 across the four plots. The following are the key takeaways:
    - With small samples (like \( n = 50 \)), the distribution of average differences is wide and rough, with a high chance of extreme values.
    - As sample size increases, the distribution becomes smoother, more symmetric, and tightly clustered around the true difference(0.004).
    - This is a clear visual demonstration of the Central Limit Theorem: even though each individual donation is binary (0 or 1), the distribution of the sample mean becomes approximately normal for large \( n \).
    - Importantly, zero is not centered in the distribution, indicating that there is a real effect and that repeated experiments would often detect it.
